In [1]:
suppressWarnings(suppressMessages(library(tidyverse)))

In [2]:
metaData <- read.table("RESULTS_FDR_LASTLEVELONLY.TAB",header=TRUE)
newData <- metaData[ which(metaData$Annotation_TC=="Conditionally_Beneficial"
                         | metaData$Annotation_TC =="Conditionally_Detrimental" |
                           metaData$Annotation_TC=="Conditionally_Essential"), ]
genes <- newData$locus_tag

In [3]:
dft <- read.table("TC_mid_batchcorrected_dclpb_included.tsv",sep = '\t',header=TRUE)%>%select(-batch,-N)%>%group_by(locus_tag,strain,condition,slevel,rep,time)
#subset
df <- dft %>% filter(locus_tag %in% newData$locus_tag)%>%summarise(mean_tncnt=mean(tncnt))%>%pivot_wider(names_from = locus_tag,values_from = mean_tncnt)
#genes
genesT <- as.data.frame(lapply(df[, 6:dim(df)[2]], unlist))

`summarise()` has grouped output by 'locus_tag', 'strain', 'condition', 'slevel', 'rep'. You can override
using the `.groups` argument.


In [4]:
condNoneSlevelNoneInd <- intersect(which(df$slevel=="none"), which(df$condition=="none"))
timeZeroInd <- which(df$time==0)
condSlevelNoneTime0 <- intersect(timeZeroInd, condNoneSlevelNoneInd)
basedfu <- df[condSlevelNoneTime0,]
basedfu2 <- basedfu%>%group_by(strain,condition,slevel)%>%summarise_all(mean)#%>%filter(strain!='DCLPB')
basedfu2$rep <- 4
basedfu3 <- basedfu2 %>% slice(rep(1:n(), each = 4))
basedfu3$rep <- rep(c(1,2,3,4), 5)
basedfufill <- rbind(basedfu,basedfu2)%>%arrange(strain,condition,slevel,rep)
dfuNoTime0 <- df[-condSlevelNoneTime0, ]
### since the base data only have three replications, we change the base level as the average
dat1 <- basedfufill%>%pivot_longer(col=-(1:5),names_to = 'gene',values_to = 'basecount')%>%mutate(Type='base',count=basecount)
dat2 <- dfuNoTime0%>%pivot_longer(col=-(1:5),names_to = 'gene',values_to = 'count')%>%mutate(Type='Condition')
dat3 <-dat1%>%full_join(dat2)%>%group_by(strain,gene,rep)%>%fill(basecount)
dat3$basecount <- dat3$basecount + 1
dat3$count <- dat3$count + 1
dat4 <-dat3%>%mutate(ratio=count/basecount)%>%filter(Type=='Condition')%>%select(-Type,-count,-basecount)
dat4$logratio <- log(dat4$ratio)
dat5 <- dat4%>%pivot_wider(id_cols = 1:5,names_from = gene,values_from = logratio)

Joining with `by = join_by(strain, condition, slevel, rep, time, gene, Type, count)`


In [5]:
wildtype <- dat5 %>%filter(strain=="wild-type")
dlon <- dat5 %>% filter(strain=="DLON")
dclpa <- dat5 %>% filter(strain=="DCLPA")
dnak <- dat5 %>% filter(strain=="dnak-dnaJ")
dclpb <- dat5 %>% filter(strain=="DCLPB")
wildMtx <- as.matrix(wildtype[ , 6:dim(wildtype)[2]])
dlonMtx <- as.matrix(dlon[, 6:dim(dlon)[2]])
dclpaMtx <- as.matrix(dclpa[, 6:dim(dclpa)[2]])
dnakMtx <- as.matrix(dnak[, 6:dim(dnak)[2]])
dclpbMtx <- as.matrix(dclpb[, 6:dim(dclpb)[2]])

In [8]:
write.csv(dat5%>%select(-time),"TC_GLM_selected.csv",row.names=FALSE,quote=FALSE)

In [14]:
metaData <- read.table("RESULTS_FDR_LASTLEVELONLY_v2.TAB",header=TRUE)
newData <- metaData[ which(metaData$Annotation_UC=="Conditionally_Beneficial"
                         | metaData$Annotation_UC =="Conditionally_Detrimental" |
                           metaData$Annotation_UC=="Conditionally_Essential"), ]
genes <- newData$locus_tag

In [15]:
dft <- read.table("UC_mid_batchcorrected_dclpb_included.tsv",sep = '\t',header=TRUE)%>%select(-batch,-N)%>%group_by(locus_tag,strain,condition,slevel,rep,time)
#subset
df <- dft %>% filter(locus_tag %in% newData$locus_tag)%>%summarise(mean_tncnt=mean(tncnt))%>%pivot_wider(names_from = locus_tag,values_from = mean_tncnt)
#genes
genesT <- as.data.frame(lapply(df[, 6:dim(df)[2]], unlist))

`summarise()` has grouped output by 'locus_tag', 'strain', 'condition', 'slevel', 'rep'. You can override
using the `.groups` argument.


In [16]:
condNoneSlevelNoneInd <- intersect(which(df$slevel=="none"), which(df$condition=="none"))
timeZeroInd <- which(df$time==0)
condSlevelNoneTime0 <- intersect(timeZeroInd, condNoneSlevelNoneInd)
basedfu <- df[condSlevelNoneTime0,]
basedfu2 <- basedfu%>%group_by(strain,condition,slevel)%>%summarise_all(mean)#%>%filter(strain!='DCLPB')
basedfu2$rep <- 4
basedfu3 <- basedfu2 %>% slice(rep(1:n(), each = 4))
basedfu3$rep <- rep(c(1,2,3,4), 5)
basedfufill <- rbind(basedfu,basedfu2)%>%arrange(strain,condition,slevel,rep)
dfuNoTime0 <- df[-condSlevelNoneTime0, ]
### since the base data only have three replications, we change the base level as the average
dat1 <- basedfufill%>%pivot_longer(col=-(1:5),names_to = 'gene',values_to = 'basecount')%>%mutate(Type='base',count=basecount)
dat2 <- dfuNoTime0%>%pivot_longer(col=-(1:5),names_to = 'gene',values_to = 'count')%>%mutate(Type='Condition')
dat3 <-dat1%>%full_join(dat2)%>%group_by(strain,gene,rep)%>%fill(basecount)
dat3$basecount <- dat3$basecount + 1
dat3$count <- dat3$count + 1
dat4 <-dat3%>%mutate(ratio=count/basecount)%>%filter(Type=='Condition')%>%select(-Type,-count,-basecount)
dat4$logratio <- log(dat4$ratio)
dat5 <- dat4%>%pivot_wider(id_cols = 1:5,names_from = gene,values_from = logratio)

Joining with `by = join_by(strain, condition, slevel, rep, time, gene, Type, count)`


In [17]:
write.csv(dat5%>%select(-time),"UC_GLM_selected.csv",row.names=FALSE,quote=FALSE)

In [18]:
dat5

strain,condition,slevel,rep,time,CCNA_00001,CCNA_00003,CCNA_00004,CCNA_00006,CCNA_00010,⋯,CCNA_R0184,CCNA_R0185,CCNA_R0187,CCNA_R0188,CCNA_R0191,CCNA_R0193,CCNA_R0195,CCNA_R0196,CCNA_R0198,CCNA_R0199
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
DCLPA,canavanine,HIGH,1,24,-2.3978953,-0.77070541,0.0000000,-0.33647224,-0.69314718,⋯,-0.5596158,-0.1541507,0.00000000,0.0000000,0.15415068,1.3862944,-0.6931472,-0.6931472,-1.2039728,0.6931472
DCLPA,canavanine,HIGH,2,24,-1.5040774,-0.26236426,0.0000000,-0.47000363,-0.81093022,⋯,-1.0986123,0.6931472,-0.69314718,0.0000000,0.00000000,-1.3862944,0.0000000,-0.6931472,-0.2231436,-0.6931472
DCLPA,canavanine,HIGH,3,24,-1.0986123,-0.46608973,0.0000000,-0.53899650,0.00000000,⋯,-0.2876821,-0.6931472,-0.18232156,0.0000000,-0.25131443,0.0000000,0.4054651,1.3862944,0.0000000,0.0000000
DCLPA,canavanine,LOW,1,24,-2.3978953,-0.11034806,0.0000000,0.06899287,-0.87546874,⋯,-0.5596158,-0.5596158,0.51082562,0.0000000,-0.18232156,0.6931472,0.0000000,-0.6931472,-0.3566749,0.0000000
DCLPA,canavanine,LOW,2,24,-1.5040774,-0.05264373,0.0000000,0.55961579,-0.40546511,⋯,-0.4054651,0.9162907,0.00000000,0.0000000,0.28768207,-0.6931472,0.6931472,0.0000000,-0.5108256,0.0000000
DCLPA,canavanine,LOW,3,24,-1.0986123,-0.40546511,0.0000000,-0.18232156,-0.18232156,⋯,0.0000000,-0.6931472,-0.69314718,0.0000000,-0.11778304,0.6931472,0.4054651,0.6931472,0.0000000,0.0000000
DCLPA,canavanine,LOW,4,24,-2.0368819,-0.52364631,0.0000000,-0.63598877,-0.40546511,⋯,-0.3483067,-0.3677248,-0.08004271,0.0000000,0.00000000,-0.6931472,-0.1541507,0.9444616,-0.2363888,-0.2876821
DCLPA,canavanine,MEDIUM,1,24,-2.3978953,-0.19735943,0.0000000,0.00000000,-0.40546511,⋯,-0.5596158,0.0000000,0.28768207,0.0000000,-0.40546511,0.6931472,-0.2876821,-0.6931472,-0.6931472,0.0000000
DCLPA,canavanine,MEDIUM,2,24,-0.8109302,-0.10821358,0.0000000,-0.28768207,-0.40546511,⋯,-0.4054651,0.9162907,0.22314355,0.0000000,0.40546511,0.0000000,0.6931472,0.0000000,-0.5108256,-0.6931472
